In [16]:
%matplotlib inline


import os
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import tensorflow.keras as keras
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense, Input, Flatten, Conv2D, BatchNormalization, \
                                    Activation, Dropout, MaxPooling2D
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import TensorBoard
from PIL import Image
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import cv2

In [2]:
data=np.load('flatland_train.npz')

In [48]:
X=data['X']
y=data['y']

In [49]:
y[y != 0] -= 2    # Correct labels so that triangle is mapped to class 1
X = X / 255

In [50]:
i=0
X_naujas=[]
for im in X:
  resized_img=cv2.resize(im,(25,25))
  X_naujas.append(resized_img)


X=np.array(X_naujas)

In [6]:
data_augmentation=keras.Sequential([
    layers.RandomTranslation(
    0,
    0.8,

)
])

In [12]:
a=data_augmentation(X)
a=a.numpy()
a.shape

(10000, 25, 25)

In [51]:
X = np.expand_dims(X, (-1))

In [52]:
X.shape

(10000, 25, 25, 1)

In [54]:
train_datagen = ImageDataGenerator(rotation_range=45,  #Too much rotation may hurt accuracy, especially for small datasets. Try 15 and 45
    width_shift_range=0.3,  #try 0.1, 0.3
    height_shift_range=0.3, #try 0.1, 0.3
    zoom_range = 0.3, #try 0.1, 0.3
)

#train_datagen.fit(X_train)

train_generator = train_datagen.flow(
    X_train,
    y_train,
    batch_size = 256)  #images to generate in a batch

In [ ]:
plt.imshow(X[5], cmap='gray')
plt.axis('off')
plt.show()

In [ ]:
plt.imshow(data_augmentation(X)[5], cmap='gray')
plt.axis('off')
plt.show()

In [53]:
X_train, X_test, y_train, y_test=train_test_split(X,y, test_size=0.3, random_state=42)

In [56]:
model = keras.models.Sequential([
    layers.Conv2D(32, kernel_size=(3, 3), input_shape=[25, 25, 1]),
    MaxPooling2D(pool_size=(2, 2),strides=1),
    layers.Conv2D(32, kernel_size=(3, 3)),
    MaxPooling2D(pool_size=(2, 2),strides=1),
    layers.Conv2D(16, kernel_size=(3, 3)),
    MaxPooling2D(pool_size=(2, 2),strides=2),
    Dropout(0.25),
    layers.Flatten(input_shape=[25, 25]),
    layers.Dense(64, activation="relu"),
    Dropout(0.2),
    layers.Dense(5, activation="softmax")])

model.compile(loss="sparse_categorical_crossentropy",
              optimizer="adam",
              metrics=["accuracy"])
model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_15 (Conv2D)          (None, 23, 23, 32)        320       
                                                                 
 max_pooling2d_15 (MaxPooli  (None, 22, 22, 32)        0         
 ng2D)                                                           
                                                                 
 conv2d_16 (Conv2D)          (None, 20, 20, 32)        9248      
                                                                 
 max_pooling2d_16 (MaxPooli  (None, 19, 19, 32)        0         
 ng2D)                                                           
                                                                 
 conv2d_17 (Conv2D)          (None, 17, 17, 16)        4624      
                                                                 
 max_pooling2d_17 (MaxPooli  (None, 8, 8, 16)         

In [ ]:
model.fit(train_generator, epochs=70, batch_size=256)

In [59]:
pred = model.predict(X_test).argmax(axis=1)
print('Accuracy on test set - {0:.02%}'.format((pred == y_test).mean()))

94/94 [==============================] - 0s 3ms/step
Accuracy on test set - 93.73%


In [61]:
model.save('model.h5')